In [ ]:
import pandas as pd
import pickle
from pathlib import Path
import json
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import os
import shutil
import seaborn as sns

In [ ]:
!mkdir -p dev/results

!wget -O dev/results/de_novo_results.zip "https://zenodo.org/records/16438770/files/de_novo_results.zip?download=1" \
    && unzip -o dev/results/de_novo_results.zip -d dev/results/ \
    && rm dev/results/de_novo_results.zip

!wget -O dev/test_data.zip "https://zenodo.org/records/16438770/files/test_datasets.zip?download=1" \
  && unzip dev/test_data.zip -d dev/ \
  && rm dev/test_data.zip

# prepare custom DB inputs for sirius 

In [ ]:
seismiq_predictions_files = [
    "dev/results/de_novo_results/52969670_seismiq_pretrained-casmi_2016.pkl",
    "dev/results/de_novo_results/52969679_seismiq_pretrained-casmi_2017.pkl",
    "dev/results/de_novo_results/52969691_seismiq_pretrained-casmi_2022.pkl",
]

dfs = []
for yr, preds in zip (["2016", "2017", "2022"], seismiq_predictions_files):
    print('preparing sirius db input for casmi', yr)

    with open(preds, "rb") as f:
        df = pickle.load(f)

    with open(f"dev/test_datasets/casmi_{yr}.json", "r") as f:
        data = json.load(f)

    df = (
        pd.DataFrame(data)
        .rename(
            columns={
                "smiles": "true_smiles",
            }
        )
        .merge(
            df,
            on=["challenge", "dataset"],
        )
    )

    df = (
        df.assign(
            # parse smiles
            true_mol=lambda d: d["true_smiles"].apply(Chem.MolFromSmiles),
            pred_mol=lambda d: d["pred_smiles"].apply(Chem.MolFromSmiles),
        ).loc[
            # exclude invalid predicted smiles
            lambda d: d["pred_mol"].notna()
        ]
        .loc[
            # exclude charged molecules - unsupported by sirius
            lambda d: d["pred_mol"].apply(lambda m: Chem.GetFormalCharge(m) == 0)
        ]
        .loc[
            # exclude molecules with the wrong sum formula
            lambda d: d["true_mol"]
            .apply(CalcMolFormula)
            .eq(d["pred_mol"].apply(CalcMolFormula))
        ]
        .assign(
            pred_smiles=lambda d: d['pred_mol'].apply(Chem.MolToSmiles),
            sirius_db_id=lambda d: d.apply(
                lambda row: f"{row['dataset']}-{row['challenge']}-pred-{row['index']}", axis=1
            )
        )
    )
    dfs.append(df)

    df[["pred_smiles", "sirius_db_id", "sirius_db_id"]].to_csv(
        f"dev/sirius_db_input-casmi_{yr}.tsv", index=False, sep="\t", header=False
    )

df = pd.concat(dfs, ignore_index=True)

# Produce SIRIUS ranking through GUI

Import the DB produced above, then import spectral data and perform structure search against that DB.

Here we download the resulting files from Zenodo

In [ ]:
!wget -O dev/results/sirius_ranking.zip "https://zenodo.org/records/16438770/files/sirius_ranking.zip?download=1" \
    && unzip -o dev/results/sirius_ranking.zip -d dev/results/ \
    && rm dev/results/sirius_ranking.zip

# Analyze SIRIUS ranking

In [ ]:
df_sirius = pd.concat(
    [
        pd.read_csv(f'dev/results/sirius_ranking/sirius_casmi_{yr}_summary/structure_identifications_all.tsv', sep="\t")
        for yr in ["2016", "2017", "2022"]
    ],
)

In [ ]:
df_sirius

In [ ]:
# canonicalize smiles
df_sirius = df_sirius.assign(
    mol=lambda d: d["smiles"].apply(Chem.MolFromSmiles),
    smiles=lambda d: d["mol"].apply(lambda m: Chem.MolToSmiles(m) if m else None),
)

In [ ]:
df_results = pd.merge(
    df,
    df_sirius,
    left_on="pred_smiles",
    right_on="smiles",
    how="left"
)

In [ ]:
df_results = (
    df_results[
        [
            "dataset",
            "challenge",
            "true_smiles",
            "pred_smiles",
            "perplexity",
            "tanimoto",
            "CSI:FingerIDScore",
            "index",
            "structurePerIdRank",
        ]
    ]
    .assign(
        perfect=lambda d: d['tanimoto'] > 0.9999
    )
)

df_results

In [ ]:
rows = []
for i, g in df_results.dropna().groupby(["dataset", "challenge"]):
    g = g.assign(
        seismiq_rank=g['perplexity'].rank(ascending=True, method='dense'),
        sirius_rank=g['CSI:FingerIDScore'].rank(ascending=False, method='dense'),
    )
    rr = pd.DataFrame([{
        'k': k,
        'seismiq_perfect': g.loc[g['seismiq_rank'] <= k, 'tanimoto'].gt(0.999).max(),
        'sirius_perfect': g.loc[g['sirius_rank'] <= k, 'tanimoto'].gt(0.999).max()
    } for k in [1, 5, 10, 25]])
    rr[['dataset', 'challenge']] = i
    rows.append(rr)


dfk = pd.concat(rows)

In [ ]:
from scipy.stats import permutation_test
import numpy as np

results = []
for dataset, group in dfk.groupby('dataset'):
    for k, gk in group.groupby('k'):
        # permutation test for difference in means
        res = permutation_test(
            (gk['seismiq_perfect'].values, gk['sirius_perfect'].values),
            statistic=lambda x, y: x.mean() - y.mean(),
            permutation_type='samples',
            n_resamples=10000,
            alternative='two-sided',
            #vectorized=True,
            random_state=82746,
        )
        results.append({
            'dataset': dataset,
            'k': k,
            'seismiq_mean': gk['seismiq_perfect'].mean(),
            'sirius_mean': gk['sirius_perfect'].mean(),
            'pvalue': res.pvalue,
            'statistic': res.statistic,
        })

df_significance = pd.DataFrame(results).assign(
    sig=lambda d: d['pvalue'].apply(
        lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else '.' if p < 0.1 else 'n.s.'
    ),
    mark=lambda d: np.where(d['pvalue'] < 0.05, "(*)", ""),
)

df_significance

In [ ]:
import matplotlib.pyplot as plt


g = dfk.melt(
    id_vars=["dataset", "challenge", "k"],
    value_vars=["sirius_perfect", "seismiq_perfect"],
).replace({
    'dataset': {
        'casmi_2016': 'CASMI 2016',
        'casmi_2017': 'CASMI 2017',
        'casmi_2022': 'CASMI 2022',
    },
    'variable': {
        'sirius_perfect': 'CSI:FingerID',
        'seismiq_perfect': 'SEISMiQ',
    }
}).rename(
    columns={
        'variable': 'Ranking',
        'value': 'Accuracy',
        'k': 'Top-k',
    }
).pipe(
    (sns.catplot, 'data'),
    x='Top-k',
    y='Accuracy',
    hue='Ranking',
    col='dataset',
    kind='bar',
    errorbar='se',
    sharey=False,
    height=3,
    aspect=1.25,
)

# Annotate significance at the top of each Top-k group, above the highest bar
for ax, dataset in zip(g.axes.flat, g.col_names):
    dataset_key = dataset.replace(' ', '_').lower()

    y_pos = max(
        bar.get_height()
        for container in ax.containers
        for bar in container
    )

    # annotate statistical significance
    for i, k in enumerate(sorted(df_significance[df_significance['dataset'] == dataset_key]['k'])):
        sig = df_significance[(df_significance['dataset'] == dataset_key) & (df_significance['k'] == k)]['sig'].values[0]
        ax.text(i, y_pos + 0.07, sig, ha='center', va='bottom', fontsize=12)
    
    ax.set_ylim(0, y_pos + 0.14)

    ax.set_title(ax.get_title().replace('dataset = ', ''))